# World Info Tuning
mkultra provides a specialized trainer for generating soft World Info*.

This trainer takes datapoints in a "call and response" format. The "call" is a fixed prompt (e.g. "A detailed description of the Spider Tank:") that should elicit the "response" (a few paragraphs of information about the Spider Tank). Multiple responses can be specified for each call, but loss is only calculated against the responses.

The trainer will optimize a soft prompt such that the call is followed by the response. The combined call and response is moved randomly within the context window to prevent overfitting, and the gap is filled with random tokens.

This setup may work for other tasks, but if your needs are significantly different then it is recommended to see tuning_finetune.ipynb for tips on rolling your own.

*Short prompt infixes for AI text adventures and writing.
Describes a character, subject, artistic direction, etc. with the intent of steering the output to consistently make use of the details in question.

In [6]:
#@title Setup for Colab only
#!pip install transformers
#!pip install git+git://github.com/corolla-johnson/mkultra.git#egg=mkultra

In [7]:
from transformers.pipelines import pipeline
from mkultra.models.tuning import GPT2PromptTuningLM
from mkultra.tokenizers import GPT2TokenizerFast
from mkultra.soft_prompt import SoftPrompt
from transformers import Adafactor
import torch
import io
import json

In [8]:
# Use an mkultra prompt tuning LM and a standard tokenizer.
model = GPT2PromptTuningLM.from_pretrained("gpt2").to("cuda")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")


In [4]:
# Set the length of the soft prompt.
# The paper does not recomment going over 100 tokens.
model.initialize_soft_prompt(n_tokens=20)

call_str = "Detailed description of Emma Violence:\n"

response_str = ("Emma Violence is a British cybernetic assassin who is known for her "
                "elegant style and high-profile targets. She is described to be a perfectionist "
                "in her work, often going above and beyond the call of duty. "
                "Despite her cold, ruthless, and calculating nature, she has a warm and "
                "motherly side that she only shows to a select few people. She has two guns "
                "implanted to her forearms and can utilize them both with deadly accuracy.")

call = tokenizer(call_str, return_tensors="pt").input_ids.cuda()
response = tokenizer(response_str, return_tensors="pt").input_ids.cuda()

input_ids = torch.cat( [call, response], dim=1 )

ignore_call = torch.full((1,call.shape[-1]),-100).cuda()

labels = torch.cat( [ ignore_call, response ], dim=1 )

print(input_ids.shape)
print(labels.shape)

# Multiple answers can be written for a single question.
# answers = [[]]

# The components are assembled like this:
# |soft prompt|random tokens|question|answer


torch.Size([1, 97])
torch.Size([1, 97])


In [5]:
#embeds = torch.load("sample_sps/world_info/emma_gpt2.pt")[0]
#print(embeds)
#model.set_soft_prompt_embeds(embeds)

In [6]:
optimizer = Adafactor(params=[model.get_soft_params()])

In [10]:
#@title Training
#@markdown 4000+ for "gpt2"
#@markdown
#@markdown 200+ for "GPT-Neo-2.7B"
model.train()
iterations = 1#@param{type:"number"}

for i in range(iterations):
  optimizer.zero_grad()
  output = model(input_ids=input_ids, labels=labels)
  loss = output.loss
  loss.backward()
  optimizer.step()
  if i%5 == 0:
    print(f"{i}: Loss: {loss}")

torch.Size([1, 2, 768])
torch.Size([1, 97, 768])
torch.Size([1, 99, 768])
torch.Size([1, 97])
torch.Size([1, 99])
0: Loss: 3.186060667037964


In [ ]:
model.eval()

basic_output = model.generate(
    call,
    do_sample=True,
    min_length=call.shape[-1] + 100,
    max_length=call.shape[-1] + 100,
    temperature=0.1,
    top_p = 0.9,
    repetition_penalty = 1.7,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(basic_output[0]))

In [ ]:
# WorldInfoTrainer will help prevent overfitting by moving the target
# up and down the context, filling the rest with random tokens.
# Also, only the 'target' string is used to calculate loss.
# Note that very low loss may not be acceptable for the description task.
"""
trainer = WorldInfoTrainer(
    model=model
    optimizer=Adafactor(params=model.get_soft_params())
    blocks = [block],
    max_spacing=0, # Setting this to 0 for illustrative purposes
    min_loss=0.4
)

trainer.train()
"""

## Best Practices for World Info
Unlike the finetuning example, this is fresh territory.
Here are potential techniques to explore:
- Define bad_words_ids (e.g. square brackets) and surround the response in them to keep the output from repeating the training data verbatim.
- Rather than describing the subject in grammatically correct sentences, try a word cloud approach.
- Rather than describing the subject, provide implicit examples of to write about it ("John doffed his black top hat and cleaned it with his white handkerchief")
- Consider the use of the subject's proper noun vs its pronouns. (Should you start every sentence with "John Doe", or a mix of "John" and "He"?)
- Definitely use the min_loss parameter to arrest the tuning.